In [1]:
!pip install -U transformers datasets accelerate optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incomp

In [4]:
import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    set_seed,
    EarlyStoppingCallback # <--- NEW: Imported for early stopping
)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# --- Global Configuration ---
set_seed(42)
MODEL_NAME = "distilbert-base-multilingual-cased"
data_path = "/content/drive/MyDrive/ITC508_data/clickbait_data.csv"
output_dir = "./grid_search_results"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# --- Load Dataset & Split ---
df = pd.read_csv(data_path)
df = df.rename(columns={"headline": "text", "clickbait": "label"})
print(f"Original Data Distribution:\n{df['label'].value_counts()}")

# Split into Train, Validation, and Test (64%, 16%, 20%)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df["label"], random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df) # <--- NEW: Test set for final evaluation

# --- Tokenization ---
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=["__index_level_0__", "text"])
val_dataset = val_dataset.map(tokenize, batched=True, remove_columns=["__index_level_0__", "text"])
test_dataset = test_dataset.map(tokenize, batched=True, remove_columns=["__index_level_0__", "text"]) # Tokenize test set

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# --- Metrics ---
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1": f1_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds),
        "recall": recall_score(p.label_ids, preds)
    }

# --- Model Initialization ---
def model_init():
    # Model is initialized on CPU and moved to device by the Trainer
    return DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

# --- Training Arguments (Base Configuration) ---
base_kwargs = {
    "output_dir": output_dir,
    "save_strategy": "epoch",
    "load_best_model_at_end": True, # Crucial for Early Stopping
    "metric_for_best_model": "f1", # Monitor F1, as accuracy is too high
    "greater_is_better": True,
    "fp16": torch.cuda.is_available(),
    "report_to": "none",
    "num_train_epochs": 10, # Increased epochs with Early Stopping
}

# Adjust for evaluation strategy naming based on library version
if hasattr(TrainingArguments, "evaluation_strategy"):
    base_kwargs["evaluation_strategy"] = "epoch"
else:
    base_kwargs["eval_strategy"] = "epoch"

# --- Hyperparameter Grid (Refined for Regularization) ---
# Focus on lower learning rates and higher weight decay to combat overfitting
search_space = [
    {"learning_rate": 5e-5, "batch_size": 8, "weight_decay": 0.0},
    {"learning_rate": 3e-5, "batch_size": 16, "weight_decay": 0.05},
    {"learning_rate": 1e-5, "batch_size": 8, "weight_decay": 0.05},
    {"learning_rate": 1e-5, "batch_size": 16, "weight_decay": 0.1}, # Strongest Regularization combo
]

best_score = 0
best_params = None
best_model_path = None
run_index = 0

# --- Grid Search Loop ---
for params in search_space:
    run_index += 1
    print(f"\n========================================================")
    print(f"🔍 Run {run_index}/{len(search_space)}: Training with: {params}")
    print(f"========================================================")

    # Update base args with specific grid parameters
    training_args = TrainingArguments(
        **base_kwargs,
        per_device_train_batch_size=params["batch_size"],
        learning_rate=params["learning_rate"],
        weight_decay=params["weight_decay"],
        # Save checkpoints for this run to a unique folder
        output_dir=f"{output_dir}/run_{run_index}",
    )

    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        # --- NEW: Add Early Stopping Callback (Patience=2 means stop if no improvement for 2 epochs) ---
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train()
    metrics = trainer.evaluate()
    print(metrics)

    # Track best model and save path
    if metrics["eval_f1"] > best_score:
        best_score = metrics["eval_f1"]
        best_params = params
        # The best model checkpoint is saved by load_best_model_at_end=True
        # We save the *actual* best model from the current run
        trainer.save_model(f"{output_dir}/best_model_final")
        best_model_path = f"{output_dir}/best_model_final"

print("\n\n" + "="*50)
print("🏆 BEST HYPERPARAMETERS FOUND:")
print(best_params)
print(f"Best Validation F1: {best_score:.4f}")
print("="*50)

# --- Final Evaluation on Test Set ---
if best_model_path:
    print("\n📝 Evaluating Best Model on Held-out Test Set...")

    # Load the best model found during the grid search
    final_model = DistilBertForSequenceClassification.from_pretrained(best_model_path).to(device)

    # Create a new trainer for evaluation only
    final_trainer = Trainer(
        model=final_model,
        args=TrainingArguments(output_dir="./test_eval", report_to="none"), # Simple args for evaluation
        compute_metrics=compute_metrics,
    )

    test_metrics = final_trainer.evaluate(test_dataset)

    print("\n\n" + "*"*50)
    print("⭐ FINAL TEST SET RESULTS ⭐")
    print(f"Best Params: {best_params}")
    print(f"Accuracy: {test_metrics['eval_accuracy']:.4f}")
    print(f"F1 Score: {test_metrics['eval_f1']:.4f}")
    print(f"Precision: {test_metrics['eval_precision']:.4f}")
    print(f"Recall: {test_metrics['eval_recall']:.4f}")
    print("*"*50)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
GPU: Tesla T4
Original Data Distribution:
label
0    16001
1    15999
Name: count, dtype: int64


Map:   0%|          | 0/20480 [00:00<?, ? examples/s]

Map:   0%|          | 0/5120 [00:00<?, ? examples/s]

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]


🔍 Run 1/4: Training with: {'learning_rate': 5e-05, 'batch_size': 8, 'weight_decay': 0.0}


...

{'eval_loss': 0.022220568731427193, 'eval_accuracy': 0.9953125, 'eval_f1': 0.995301487862177, 'eval_precision': 0.9976452119309263, 'eval_recall': 0.99296875, 'eval_runtime': 6.1557, 'eval_samples_per_second': 831.744, 'eval_steps_per_second': 103.968, 'epoch': 4.0}

🔍 Run 4/4: Training with: {'learning_rate': 1e-05, 'batch_size': 16, 'weight_decay': 0.1}


/tmp/ipython-input-3423641199.py:133: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.033800,0.022945,0.994141,0.994147,0.992985,0.995313
2,0.018600,0.026893,0.994141,0.994161,0.990690,0.997656
3,0.003800,0.049853,0.992773,0.992727,0.999209,0.986328
4,0.002300,0.037623,0.994531,0.994553,0.990698,0.998437
5,0.002600,0.033258,0.994922,0.994924,0.994536,0.995313
6,0.000800,0.028010,0.996289,0.996294,0.994936,0.997656
7,0.002200,0.038234,0.995703,0.995713,0.993390,0.998047
8,0.000000,0.039961,0.995313,0.995323,0.993002,0.997656


{'eval_loss': 0.028010064736008644, 'eval_accuracy': 0.9962890625, 'eval_f1': 0.9962941291203433, 'eval_precision': 0.9949357226334242, 'eval_recall': 0.99765625, 'eval_runtime': 6.306, 'eval_samples_per_second': 811.929, 'eval_steps_per_second': 101.491, 'epoch': 8.0}


🏆 BEST HYPERPARAMETERS FOUND:
{'learning_rate': 3e-05, 'batch_size': 16, 'weight_decay': 0.05}
Best Validation F1: 0.9971

📝 Evaluating Best Model on Held-out Test Set...




**************************************************
⭐ FINAL TEST SET RESULTS ⭐
Best Params: {'learning_rate': 3e-05, 'batch_size': 16, 'weight_decay': 0.05}
Accuracy: 0.9934
F1 Score: 0.9935
Precision: 0.9907
Recall: 0.9962
**************************************************
